In [1]:
import sys; sys.path.append('../../wwa'); from wwa import *

## Extract gridded data for model fitting

In [3]:
fl = glob.glob("../../00_WWA_project_folder/ephemeral/Asia_Heatwave/obs/obs_fields/*.nc")

domain = {"EA" : [118,130,4,20], "WA" : [30,50,25,40]}

In [ ]:
for fnm in glob.glob("../../00_WWA_project_folder/ephemeral/Asia_Heatwave/obs/obs_fields/*.nc")[:1]:
    
    xn,xx,yn,yx = domain[fnm[-5:-3]]
    
    da = xr.open_dataset(fnm).sel(lon = slice(xn,xx), lat = slice(yn,yx))
    da = da.assign_coords(time = da.time.dt.year).rename(time = "year")

    if "mswx" in fnm:
        da = da.air_temperature
    else:
        da = da.tmax
    
    # mask off the sea
    rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
    da = da.where(rm == 0)
    
    # filter any cells with only partial data
    da = da.where(np.isnan(da).sum("year") == 0)
    
    # flatten & save as .csv for transfer to R
    da.stack(xy = ["lat", "lon"]).dropna("xy", "all").to_pandas().to_csv("data/"+fnm.split("/")[-1].split("_")[0]+"_"+fnm[-5:-3]+"_flattened.csv")
    
    # save map for easier reconstruction
    da.mean("year").to_netcdf("data/"+fnm.split("/")[-1].split("_")[0]+"_tmplt.nc")

<xarray.DataArray 'tmax' (year: 46, lat: 30, lon: 40)>
array([[[42.236694, 42.920452, 41.858723, ..., 38.66435 , 38.957397,
         39.161747],
        [41.26556 , 41.768047, 41.245472, ..., 38.584152, 38.758083,
         38.97155 ],
        [40.699543, 40.803883, 40.667805, ..., 38.530514, 38.581806,
         38.626762],
        ...,
        [25.123999, 25.653572, 25.469336, ..., 20.22464 ,       nan,
               nan],
        [25.555107, 25.771154, 26.972902, ..., 22.212423,       nan,
               nan],
        [26.921988, 27.441023, 26.77268 , ..., 22.739845, 21.083605,
               nan]],

       [[42.69865 , 42.731388, 41.180946, ..., 39.51994 , 39.665768,
         39.627354],
        [41.522747, 41.817158, 41.044613, ..., 39.327923, 39.19174 ,
         39.29935 ],
        [40.687298, 40.72591 , 40.58316 , ..., 38.97691 , 38.883526,
         38.920334],
...
        [19.013899, 19.39148 , 19.222563, ..., 22.706032,       nan,
               nan],
        [19.340282, 19.010439, 20.22875 , ..., 25.308334,       nan,
               nan],
        [20.829805, 20.202026, 20.083725, ..., 27.351067, 26.48256 ,
               nan]],

       [[41.963356, 42.34856 , 41.343037, ..., 38.416553, 38.984554,
         38.57743 ],
        [40.871246, 41.466106, 41.211746, ..., 37.880417, 37.75869 ,
         37.12096 ],
        [40.81552 , 41.144543, 41.27593 , ..., 36.995426, 36.366154,
         35.706913],
        ...,
        [26.687464, 27.087831, 26.868505, ..., 24.5792  ,       nan,
               nan],
        [27.394188, 27.357681, 28.480066, ..., 27.173574,       nan,
               nan],
        [28.229774, 28.589874, 28.17452 , ..., 28.944822, 27.830465,
               nan]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 25.25 25.75 26.25 26.75 ... 38.25 38.75 39.25 39.75
  * lon      (lon) float32 30.25 30.75 31.25 31.75 ... 48.25 48.75 49.25 49.75
  * year     (year) int64 1979 1980 1981 1982 1983 ... 2020 2021 2022 2023 2024
Attributes: (12/13)
    level_desc:    Surface
    parent_stat:   Other
    long_name:     Daily Maximum Temperature
    cell_methods:  time: mean
    valid_range:   [-90.  50.]
    avg_period:    0000-00-01 00:00:00
    ...            ...
    comments:      GTS data and is gridded using the Shepard Algorithm
    max_period:    6z to 6z
    actual_range:  [-59.656933  22.819536]
    units:         degC
    var_desc:      Maximum Temperature
    statistic:     Maximum